In [1]:
import pm4py
pm4py.__version__

'1.3.0'

# Social Network Analysis

> Discover the organizational perspective of the process. For each of the following networks, try to find a clear organizational structure and discuss the structure obtained. If no clear structure is to be found, explain why this is the case.

## Data loading

Import the log.

In [10]:
import os
import pandas as pd
from pm4py.objects.conversion.log import factory as log_converter
from pm4py.util import constants

PROJ_ROOT = os.path.abspath(os.path.pardir)
data_path = os.path.join(PROJ_ROOT, 'data', 'log.csv')

assert os.path.exists(data_path)

df_log = pd.read_csv(data_path)

# map dataset columns to PM4Py keys
param_keys={constants.PARAMETER_CONSTANT_CASEID_KEY: 'Patient',
            constants.PARAMETER_CONSTANT_RESOURCE_KEY: 'Resource', 
            constants.PARAMETER_CONSTANT_ACTIVITY_KEY: 'Activity',
            constants.PARAMETER_CONSTANT_TIMESTAMP_KEY: 'Timestamp',
            constants.PARAMETER_CONSTANT_START_TIMESTAMP_KEY: 'start_timestamp'}

event_log = log_converter.apply(df_log, parameters=param_keys)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:20: DeprecatedWarning: apply is deprecated as of 1.3.0 and will be removed in 2.0.0. Use algorithm entrypoint instead


---

# a)
> Handover-of-Work Social Network

In [16]:
# define the path where the resulting networks will be stored
RESULT_PATH = os.path.join(PROJ_ROOT, 'results/Q2/')

In [17]:
from pm4py.algo.enhancement.sna import factory as sna_factory
from pm4py.visualization.sna import factory as sna_vis_factory

#discover the network
handover_net = sna_factory.apply(event_log, parameters=param_keys, variant="handover")

#create a visualization of the network
handover_viz = sna_vis_factory.apply(handover_net, variant="pyvis")

#save the visualization of the discovered network
sna_vis_factory.save(handover_viz, RESULT_PATH+"handover_net.html", variant="pyvis")


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:4: DeprecatedWarning: apply is deprecated as of 1.3.0 and will be removed in 2.0.0. Use algorithm entrypoint instead
  after removing the cwd from sys.path.
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecatedWarning: apply is deprecated as of 1.3.0 and will be removed in 2.0.0. Use visualizer module instead.
  import sys
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:10: DeprecatedWarning: save is deprecated as of 1.3.0 and will be removed in 2.0.0. Use visualizer module instead.
  # Remove the CWD from sys.path while we load stuff.


As the pyvis interactive visualization cannot be embedded, the discovered network is shown below in the form of an image. The real network can be found in the results/Q2 folder.

![](../report/figures/q2_handover.png)

---

# b)
> Subcontracting Social Network

In [18]:
#discover the network
subcontracting_net = sna_factory.apply(event_log, parameters=param_keys, variant="subcontracting")

#create a visualization of the network
subcontracting_viz = sna_vis_factory.apply(subcontracting_net, variant="pyvis")

#save the visualization of the discovered network
sna_vis_factory.save(subcontracting_viz, RESULT_PATH+"subcontracting_net.html", variant="pyvis")

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecatedWarning: apply is deprecated as of 1.3.0 and will be removed in 2.0.0. Use algorithm entrypoint instead
  
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecatedWarning: apply is deprecated as of 1.3.0 and will be removed in 2.0.0. Use visualizer module instead.
  """
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:8: DeprecatedWarning: save is deprecated as of 1.3.0 and will be removed in 2.0.0. Use visualizer module instead.
  


As the pyvis interactive visualization cannot be embedded, the discovered network is shown below in the form of an image. The real network can be found in the results/Q2 folder.

![](../report/figures/q2_subcontracting.png)

---

# c)
> Working-Together Social Network

In [19]:
#discover the network
working_together_net = sna_factory.apply(event_log, parameters=param_keys, variant="working_together")

#create a visualization of the network
working_together_viz = sna_vis_factory.apply(working_together_net, variant="pyvis")

#save the visualization of the discovered network
sna_vis_factory.save(working_together_viz, RESULT_PATH+"working_together_net.html", variant="pyvis")

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecatedWarning: apply is deprecated as of 1.3.0 and will be removed in 2.0.0. Use algorithm entrypoint instead
  
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecatedWarning: apply is deprecated as of 1.3.0 and will be removed in 2.0.0. Use visualizer module instead.
  """
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:8: DeprecatedWarning: save is deprecated as of 1.3.0 and will be removed in 2.0.0. Use visualizer module instead.
  


As the pyvis interactive visualization cannot be embedded, the discovered network is shown below in the form of an image. The real network can be found in the results/Q2 folder.

![](../report/figures/q2_working_together.png)

--- 

# d)
> Joint-Activity Network

In [20]:
#discover the network
jointactivities_net = sna_factory.apply(event_log, parameters=param_keys, variant="jointactivities")

#create a visualization of the network
jointactivities_viz = sna_vis_factory.apply(jointactivities_net, variant="pyvis")

#save the visualization of the discovered network
sna_vis_factory.save(jointactivities_viz, RESULT_PATH+"jointactivities_net.html", variant="pyvis")

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecatedWarning: apply is deprecated as of 1.3.0 and will be removed in 2.0.0. Use algorithm entrypoint instead
  
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecatedWarning: apply is deprecated as of 1.3.0 and will be removed in 2.0.0. Use visualizer module instead.
  """
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:8: DeprecatedWarning: save is deprecated as of 1.3.0 and will be removed in 2.0.0. Use visualizer module instead.
  


As the pyvis interactive visualization cannot be embedded, the discovered network is shown below in the form of an image. The real network can be found in the results/Q2 folder.

![](../report/figures/q2_jointactivities.png)